# TRAORE Koudadim Olivier

## TP N°2 : Calcul de la Value at Risk (VaR) d'un contrat à terme sur devises

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("CAD_USD.xlsx")
print(data)

    Unnamed: 0   R.CAN   R.USA  USD_CAD
0            1  2.4906  2.0732   1.2504
1            2  2.5009  2.0765   1.2868
2            3  2.5017  2.0718   1.3426
3            4  2.5859  2.0859   1.3469
4            5  2.4939  2.0703   1.3504
5            6  2.4902  2.0226   1.3678
6            7  2.5278  2.0833   1.3568
7            8  2.4912  2.0542   1.3785
8            9  2.4127  2.0842   1.3973
9           10  2.4336  2.0798   1.4131
10          11  2.4122  2.0748   1.3514
11          12  2.4109  2.0352   1.3691
12          13  2.4108  1.9962   1.3436
13          14  2.4074  2.0199   1.3297
14          15  2.4138  2.0632   1.2822
15          16  2.3988  2.0030   1.3108
16          17  2.4103  1.9642   1.3730
17          18  2.4504  1.9732   1.3953
18          19  2.4457  1.9285   1.3843
19          20  2.4665  1.9449   1.3203
20          21  2.4858  2.0057   1.3456
21          22  2.5475  1.9707   1.3217
22          23  2.5792  2.0323   1.3217
23          24  2.6175  1.9949   1.3739


In [3]:
df = data[['R.CAN', 'R.USA', 'USD_CAD']]
print(df)

     R.CAN   R.USA  USD_CAD
0   2.4906  2.0732   1.2504
1   2.5009  2.0765   1.2868
2   2.5017  2.0718   1.3426
3   2.5859  2.0859   1.3469
4   2.4939  2.0703   1.3504
5   2.4902  2.0226   1.3678
6   2.5278  2.0833   1.3568
7   2.4912  2.0542   1.3785
8   2.4127  2.0842   1.3973
9   2.4336  2.0798   1.4131
10  2.4122  2.0748   1.3514
11  2.4109  2.0352   1.3691
12  2.4108  1.9962   1.3436
13  2.4074  2.0199   1.3297
14  2.4138  2.0632   1.2822
15  2.3988  2.0030   1.3108
16  2.4103  1.9642   1.3730
17  2.4504  1.9732   1.3953
18  2.4457  1.9285   1.3843
19  2.4665  1.9449   1.3203
20  2.4858  2.0057   1.3456
21  2.5475  1.9707   1.3217
22  2.5792  2.0323   1.3217
23  2.6175  1.9949   1.3739
24  2.8378  1.8744   1.3708


### Données

On a les données suivantes :
    
Contrat à terme de maturité T trois (3) mois permettant l'achat à terme d'un millions USD contre 1.59 millions CAD.  

La valeur du contrat est donnée par :

f(t) = S(t) / ( 1+R.USA(t)) - K / ( 1+R.CAN(t))

Où : 

S(t) : prix au comptant du USD en CAD

K : Taux de change du dollar américain spécifié dans le contrat, ici, K = 1.59

R.USA(t) et R.CAN : taux d'intérêt sans risque américain et canadien

A la date de calcul de la VaR :

R.USD = 2.75%

R.CAN = 3.31%

S = 1.588 c-à-d : 1 USD = 1.588 CAD

### 1. Calculons les variations journalières des données

In [4]:
R_CAN = 0.0331
R_USA = 0.0275
USD_CAD = 1.588

Delta_R_CAN = [] # liste des variations journalières du taux d'intérêt canadien
Delta_R_USA = []  # liste des variations journalières du taux d'intérêt américain
Delta_USD_CAD = [] # liste des variations journalières du taux de change

for i in range(len(df.index)) :
    Delta_R_CAN.append(df['R.CAN'][i]*0.01 - R_CAN)  # On multiplie par 0.01 car les taux sont en exprimés en %
    Delta_R_USA.append(df['R.USA'][i]*0.01 -R_USA)
    Delta_USD_CAD.append(df['USD_CAD'][i]-USD_CAD)

# Un dataframe des variations créer à partir du dictionnaire des variations

Dict = {
    'Delta_R_CAN' : Delta_R_CAN,
    'Delta_R_USA' : Delta_R_USA,
    'Delta_USD_CAD' : Delta_USD_CAD
}

Variations = pd.DataFrame(Dict)
print(Variations)

    Delta_R_CAN  Delta_R_USA  Delta_USD_CAD
0     -0.008194    -0.006768        -0.3376
1     -0.008091    -0.006735        -0.3012
2     -0.008083    -0.006782        -0.2454
3     -0.007241    -0.006641        -0.2411
4     -0.008161    -0.006797        -0.2376
5     -0.008198    -0.007274        -0.2202
6     -0.007822    -0.006667        -0.2312
7     -0.008188    -0.006958        -0.2095
8     -0.008973    -0.006658        -0.1907
9     -0.008764    -0.006702        -0.1749
10    -0.008978    -0.006752        -0.2366
11    -0.008991    -0.007148        -0.2189
12    -0.008992    -0.007538        -0.2444
13    -0.009026    -0.007301        -0.2583
14    -0.008962    -0.006868        -0.3058
15    -0.009112    -0.007470        -0.2772
16    -0.008997    -0.007858        -0.2150
17    -0.008596    -0.007768        -0.1927
18    -0.008643    -0.008215        -0.2037
19    -0.008435    -0.008051        -0.2677
20    -0.008242    -0.007443        -0.2424
21    -0.007625    -0.007793    

### 2. Le valeurs simulées du contrat à terme

In [5]:
f = []
K = 1.59
for t in range(len(df.index)) :
    val = df['USD_CAD'][t] / (1+ (t+1)*(df['R.USA'][t]*0.01) ) - K / ( 1+(t+1)*(df['R.CAN'][t]*0.01) )
    f.append(val)
    
d = {
    'Jours' : [i+1 for i in range(len(df.index))],
    'Valeurs' : f
}

Valeur = pd.DataFrame(d)
print('Les valeurs simulées du contrat sont:')
print(Valeur)


Les valeurs simulées du contrat sont:
    Jours   Valeurs
0       1 -0.326359
1       2 -0.278770
2       3 -0.214964
3       4 -0.197779
4       5 -0.189991
5       6 -0.163543
6       7 -0.166835
7       8 -0.141841
8       9 -0.129741
9      10 -0.108989
10     11 -0.156293
11     12 -0.132855
12     13 -0.143827
13     14 -0.152626
14     15 -0.188174
15     16 -0.156334
16     17 -0.098557
17     18 -0.073737
18     19 -0.072470
19     20 -0.114202
20     21 -0.097858
21     22 -0.096963
22     23 -0.097291
23     24 -0.047459
24     25  0.003282


### 3. Rangeons les valeurs simulées par ordre coissant et donnons la distribution de la VaR

In [6]:
PnL =Valeur['Valeurs']
print('Les profits and loss journaliers\n')
print(PnL)

Les profits and loss journaliers

0    -0.326359
1    -0.278770
2    -0.214964
3    -0.197779
4    -0.189991
5    -0.163543
6    -0.166835
7    -0.141841
8    -0.129741
9    -0.108989
10   -0.156293
11   -0.132855
12   -0.143827
13   -0.152626
14   -0.188174
15   -0.156334
16   -0.098557
17   -0.073737
18   -0.072470
19   -0.114202
20   -0.097858
21   -0.096963
22   -0.097291
23   -0.047459
24    0.003282
Name: Valeurs, dtype: float64


In [7]:
PnL_sorted = PnL.sort_values()
print('Les PnL classés par ordre croissant\n')
print(PnL_sorted)

Les PnL classés par ordre croissant

0    -0.326359
1    -0.278770
2    -0.214964
3    -0.197779
4    -0.189991
14   -0.188174
6    -0.166835
5    -0.163543
15   -0.156334
10   -0.156293
13   -0.152626
12   -0.143827
7    -0.141841
11   -0.132855
8    -0.129741
19   -0.114202
9    -0.108989
16   -0.098557
20   -0.097858
22   -0.097291
21   -0.096963
17   -0.073737
18   -0.072470
23   -0.047459
24    0.003282
Name: Valeurs, dtype: float64


In [8]:
n = 25 # Nombre de jours
alpha = 0.05
p = n*alpha
q = int(p)
Value_at_Risk = abs( PnL_sorted[q]  + ( p-q)* (PnL_sorted[q+1]- PnL_sorted[q]) ) 
print(Value_at_Risk)

0.2628183549862361


In [9]:
df

# Tableau de variations


,R.CAN,R.USA,USD_CAD
0,2.4906,2.0732,1.2504
1,2.5009,2.0765,1.2868
2,2.5017,2.0718,1.3426
3,2.5859,2.0859,1.3469
4,2.4939,2.0703,1.3504
5,2.4902,2.0226,1.3678
6,2.5278,2.0833,1.3568
7,2.4912,2.0542,1.3785
8,2.4127,2.0842,1.3973
9,2.4336,2.0798,1.4131
